In [0]:

import dlt
from pyspark.sql.functions import sum, count, round, col, desc

# --- CONFIGURATION ---
# Source tables from the physical Silver schema
SOURCE_CATALOG = "adb_catalog"
SOURCE_SCHEMA  = "silver"

# -- CUSTOMER SALES SUMMARY --
# Business Logic: Aggregates order history to provide a 360-degree view of customer value.

@dlt.materialized_view(
    name="customer_sales_summary",
    comment="Executive dashboard table providing key performance indicators (KPIs) per customer.",
    table_properties={
        "quality": "gold",
        "layer": "reporting",
        "owner": "sales_analytics_team"
    }
)
def customer_sales_summary():

    # 1. Read from the physical Silver schema
    customers = spark.read.table(f"{SOURCE_CATALOG}.{SOURCE_SCHEMA}.customers_silver")
    orders = spark.read.table(f"{SOURCE_CATALOG}.{SOURCE_SCHEMA}.orders_silver")
    
    # 2. Join and Aggregate
    return (
        orders.join(customers, "customerId", "inner")
        .groupBy("customerId", "Name", "email")
        .agg(
            round(sum("order_amount"), 2).alias("total_revenue"),
            count("orderId").alias("order_count"),
            round(sum("order_amount") / count("orderId"), 2).alias("avg_transaction_value")
        )
        .select(
            col("customerId").alias("customer_id"),
            col("Name").alias("customer_name"),
            col("email"),
            col("total_revenue"),
            col("order_count"),
            col("avg_transaction_value")
        )
    )